In [1]:
import networkx as nx
from tqdm import tqdm
from datetime import datetime
import numpy as np
import random

In [2]:
file = open('ca-AstroPh.txt','r')
lines = file.readlines()

In [3]:
def dic2list(myDic):
    graph = []
    for i in myDic.keys():
        graph.append(i)
    return graph

In [4]:
nodes = {}
unique_nodes = set()
neighbor_c = {}
for l in lines:
    if '#' in str(l):
        continue
    node = l.replace('\n','').split('\t')
    n1 = int(node[0])
    n2 = int(node[1])
    nodes[(n1,n2)] = 1
    unique_nodes.add(n1)
    unique_nodes.add(n2)
    if  neighbor_c.get(n1,0) == 0:
             neighbor_c[n1] = []
    neighbor_c[n1].append(n2)
nodes_set = dic2list(nodes)

In [48]:
def reward(visited, queue, graph, node,out):
    visited.append(node)
    queue.append(node)
    i = 1
    while queue:
        s = queue.pop(0)
        if s == out:
            break
        i+=1
        # for neighbour in graph[s]:
        if graph.get(s,0) != 0:
           for neighbour in graph.get(s,0):
                if neighbour not in visited:
                    visited.append(neighbour)
                    queue.append(neighbour)
    return i

In [6]:
def iter_bfs(neigh,neighbor_probab):
    visited_n = []
    queue_n = []
    n_neighbor = {}
    for a in tqdm(neigh):
        n_neighbor[a] = reward(queue_n, visited_n,neighbor_probab,a)
    return n_neighbor

In [7]:
# iter_bfs(unique_nodes,neighbor_c)

In [8]:
def generate_realization_graph(nodes,p):
    neighbor_probable = {}
    nodes_probable_unique = set()
    nodes_probable =  nodes.copy()
    for i in list(nodes_probable.keys()):
        if np.random.uniform(0,1,1)[0] > p:
            del nodes_probable[i]
        else:
            nodes_probable_unique.add(i[0])
            nodes_probable_unique.add(i[1])
            if  neighbor_probable.get(i[0],0) == 0:
                 neighbor_probable[i[0]] = []
            if  neighbor_probable.get(i[1],0) == 0:
                 neighbor_probable[i[1]] = []
            neighbor_probable[i[0]].append(i[1])

    return nodes_probable,nodes_probable_unique,neighbor_probable

# nod = {(5,3):1,(5,7):1,(3,2):1,(3,4):1,(7,8):1,(4,8):1}
# x = generate_realization_graph(nod,1)[2]

In [9]:
# nod = {(5,3):1,(5,7):1,(3,2):1,(3,4):1,(7,8):1,(4,8):1}
# x = generate_realization_graph(nod,1)[2]

In [10]:

# my_bfs([],[],x,5,8)[1]

In [11]:
nodes_p,nodes_p_unique,neighbor_p =  generate_realization_graph(nodes,0.015)
len(nodes_p)

5875

In [40]:
r_nodes_graph = list(nodes_p_unique)
random.shuffle(r_nodes_graph)
outbreak = r_nodes_graph[0:10]
print(type(outbreak))

<class 'list'>


In [60]:
rewards = {}
def calculate_node_reward( ):
    for n in tqdm(nodes_p_unique):
        n_list = []
        for o in outbreak:
             n_list.append(reward([],[],neighbor_p,o,n))
        rewards[n] = np.sum(n_list)

calculate_node_reward( )
print(rewards)


100%|██████████| 6470/6470 [00:38<00:00, 168.46it/s]

{3: 813, 32776: 813, 65546: 813, 98315: 813, 98318: 813, 131091: 813, 32787: 813, 98325: 813, 32791: 813, 30: 813, 32799: 813, 35: 813, 98345: 813, 43: 813, 131121: 813, 65595: 813, 32840: 813, 84: 813, 86: 813, 65629: 813, 131167: 813, 65637: 813, 65642: 514, 131180: 813, 98412: 813, 98417: 813, 114: 813, 117: 813, 119: 813, 131204: 813, 98441: 813, 65676: 813, 131221: 813, 155: 813, 164: 813, 98471: 813, 172: 637, 65713: 813, 65723: 813, 189: 813, 98506: 813, 98511: 708, 131281: 813, 65751: 813, 65753: 813, 229: 813, 131303: 813, 98541: 813, 98545: 813, 131315: 813, 65794: 813, 65795: 813, 65801: 813, 98571: 813, 33040: 743, 276: 813, 131352: 407, 98589: 813, 65822: 813, 289: 813, 291: 813, 65829: 813, 293: 813, 131370: 215, 131371: 813, 304: 813, 131385: 813, 98621: 813, 131391: 813, 98629: 813, 98631: 813, 33101: 813, 98639: 797, 131409: 813, 131413: 813, 342: 813, 98647: 813, 346: 813, 98651: 813, 98654: 813, 65889: 813, 33126: 813, 65895: 813, 33139: 813, 131451: 544, 386: 813, 6

In [50]:
sorted(rewards, key=lambda k: rewards[k], reverse=True)[0:10]

[3, 32776, 65546, 98315, 98318, 131091, 32787, 98325, 32791, 30]

### Greedy

In [39]:
def calculate_node_reward(nodes_p_unique,neighbor_p,outbreak ):
    reward_new = {}
    for n in tqdm(nodes_p_unique):
        n_list = []
        for o in outbreak:
             n_list.append(-1 * reward([],[],neighbor_p,o,n))
        reward_new[n] = np.sum(n_list)
    return reward_new



In [49]:
def hill_climing():
    S = []
    S.append(sorted(rewards, key=lambda k: rewards[k], reverse=True)[0])
    unique_nodes_h = nodes_p_unique.copy()
    outbreak_h = outbreak.copy()
    neighbor_p_c = neighbor_p.copy()
    outbreak.pop(0)
    neigh = set()
    for s in S:
        unique_nodes_h.remove(s)
        del neighbor_p_c[s]
    while len(S) < 4:
        temp_dic = {}
        temp_dic.update(calculate_node_reward(unique_nodes_h,neighbor_p_c,outbreak_h))
        n = sorted(temp_dic, key=lambda k: temp_dic[k], reverse=True)[0]
        S.append(n)
        unique_nodes_h.remove(n)
        del neighbor_p_c[n]
        outbreak.pop(0)
    return S
print(hill_climing())

100%|██████████| 6467/6467 [00:00<00:00, 34160.37it/s]

[3, 71072, 109733, 114255]


In [59]:
len(list(nx.bfs_tree(nx.DiGraph(dic2list(nodes_p)),3  )))

2

### Lazy Hill Climbing

In [ ]:
def lazy_